In [8]:
!pip install kaggle nltk scikit-learn pandas numpy matplotlib seaborn


[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install transformers datasets torch --quiet



[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
!kaggle datasets download -d shivamkushwaha/bbc-full-text-document-classification --quiet

Traceback (most recent call last):
  File "C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\lenovo\AppData\Local\Programs\Python\Python310\Scripts\kaggle.exe\__main__.py", line 7, in <module>
  File "C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\kaggle\cli.py", line 68, in main
    out = args.func(**command_args)
  File "C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\kaggle\api\kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
  File "C:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\kaggle\api\kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
KeyError: 'us

In [11]:
import os
import zipfile
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
import zipfile
import os

print("📦 Extracting dataset from local file...")

# Path to your zip file
zip_path = r"C:\Users\lenovo\Desktop\archive (1).zip"

# Where to extract
extract_to = "bbc_dataset"  # creates folder in current working directory
os.makedirs(extract_to, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_to)

print("✅ Extraction complete! Files are in:", extract_to)


📦 Extracting dataset from local file...
✅ Extraction complete! Files are in: bbc_dataset


#Step 1 — Find the BBC root folder
--------------------------------------------------------

In [13]:
import os
import pandas as pd

# These are the 5 categories in the dataset
CATEGORIES = ['business', 'entertainment', 'politics', 'sport', 'tech']

# This is where you extracted the zip in your previous cell
BASE_DIR = "bbc_dataset"   # keep as-is unless you extracted elsewhere

def find_bbc_root(base_dir):
    for root, dirs, files in os.walk(base_dir):
        if set(CATEGORIES).issubset(set(dirs)):
            return root
    raise FileNotFoundError(
        f"Could not find category folders under {base_dir}. "
        f"Make sure the extracted structure contains subfolders: {CATEGORIES}"
    )

bbc_root = find_bbc_root(BASE_DIR)
print("✅ Found BBC root folder:", bbc_root)


✅ Found BBC root folder: bbc_dataset\bbc


Read all text files into a DataFrame
----------------------------------------

In [14]:
rows = []

for cat in CATEGORIES:
    cat_dir = os.path.join(bbc_root, cat)
    for fname in os.listdir(cat_dir):
        fpath = os.path.join(cat_dir, fname)
        if os.path.isfile(fpath):
            with open(fpath, "r", encoding="utf-8", errors="ignore") as f:
                text = f.read()
            rows.append({
                "category": cat,
                "text": text
            })

df = pd.DataFrame(rows)
print("✅ Dataset loaded! Shape:", df.shape)
df.head()


✅ Dataset loaded! Shape: (2225, 2)


,category,text
0,business,Ad sales boost Time Warner profit\n\nQuarterly...
1,business,Dollar gains on Greenspan speech\n\nThe dollar...
2,business,Yukos unit buyer faces loan claim\n\nThe owner...
3,business,High fuel prices hit BA's profits\n\nBritish A...
4,business,Pernod takeover talk lifts Domecq\n\nShares in...


Save a CSV for fast reloading later
----------------------------------------

In [15]:
df[['category','text']].to_csv("bbc_text.csv", index=False, encoding="utf-8")
print("💾 Saved to bbc_text.csv")


💾 Saved to bbc_text.csv


preprocessing the text
---------------------------------------

In [16]:
# Make sure resources are downloaded
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# We assume df already exists from your BBC dataset loading step
# (df with columns: category, text)

# Initialize tools
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(tokens)

# Apply cleaning and store in new column
df['clean_text'] = df['text'].apply(clean_text)

print("✅ Text cleaning complete!")
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


✅ Text cleaning complete!


,category,text,clean_text
0,business,Ad sales boost Time Warner profit\n\nQuarterly...,ad sale boost time warner profit quarterly pro...
1,business,Dollar gains on Greenspan speech\n\nThe dollar...,dollar gain greenspan speech dollar hit highes...
2,business,Yukos unit buyer faces loan claim\n\nThe owner...,yukos unit buyer face loan claim owner embattl...
3,business,High fuel prices hit BA's profits\n\nBritish A...,high fuel price hit ba profit british airway b...
4,business,Pernod takeover talk lifts Domecq\n\nShares in...,pernod takeover talk lift domecq share uk drin...


data splitting step
-------------------------------------

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Example: assuming you have a DataFrame 'df' with columns 'text' and 'label'
X = df['text']       # Features (the text data)
y = df['category']      # Target labels

# 1. Split into train and test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 2. Apply TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)



Baseline Model (Algorithms & Models)
-------------------------------------------

1. Convert Text to TF-IDF Vectors
-------------------------------

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming your cleaned text is in 'X_train' and 'X_test'
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


2. Train a Baseline Classifier
------------------------------------------

In [19]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_tfidf, y_train)


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


3. Evaluate Model Performance
-------------------------

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

# Predict
y_pred = lr_model.predict(X_test_tfidf)  # or nb_model.predict(...)

# Metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))

# Confusion Matrix & Detailed Report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9640449438202248
Precision: 0.964169437114019
Recall: 0.9640449438202248
[[112   0   2   0   1]
 [  1  68   1   0   2]
 [  2   0  72   1   1]
 [  1   0   0 101   0]
 [  2   2   0   0  76]]
               precision    recall  f1-score   support

     business       0.95      0.97      0.96       115
entertainment       0.97      0.94      0.96        72
     politics       0.96      0.95      0.95        76
        sport       0.99      0.99      0.99       102
         tech       0.95      0.95      0.95        80

     accuracy                           0.96       445
    macro avg       0.96      0.96      0.96       445
 weighted avg       0.96      0.96      0.96       445



NLP Upgrade
----------------------------------

Prepare the dataset for Transformers
----------------------------------------

In [21]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch

# Map category names to numeric labels
labels_map = {label: idx for idx, label in enumerate(df['category'].unique())}
df['label'] = df['category'].map(labels_map)

# Split into train/test
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Convert pandas → Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df[['clean_text', 'label']])
test_dataset = Dataset.from_pandas(test_df[['clean_text', 'label']])


Load DistilBERT tokenizer & model
---------------------------------------

In [22]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_map))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenize the text
-------------------------------

In [23]:
def tokenize(batch):
    return tokenizer(batch['clean_text'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

# Set format for PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


Map:   0%|          | 0/1780 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

Fine-tune the model
---------------------------------

In [24]:
from transformers import Trainer, TrainingArguments


In [25]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50
)



In [26]:
pip install --upgrade transformers


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
!pip install 'accelerate>=0.26.0

ERROR: Invalid requirement: "'accelerate"

[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
!pip install transformers[torch]


[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
    from transformers import Trainer

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer
    )

    # Start training
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
    print(eval_results)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_5296\2881813003.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
50,0.865600
100,0.174000
150,0.087400
200,0.066400
250,0.048600


Compare results with the baseline model.
---------------------------------------------------

In [ ]:
from torch.utils.data import DataLoader
import torch

# Make sure we're using the right device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Create DataLoader from test_dataset
test_loader = DataLoader(test_dataset, batch_size=16)

y_true_bert = []
y_pred_bert = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        y_true_bert.extend(labels.cpu().numpy())
        y_pred_bert.extend(predictions.cpu().numpy())

# Metrics
accuracy_bert = accuracy_score(y_true_bert, y_pred_bert)
f1_bert = f1_score(y_true_bert, y_pred_bert, average='weighted')

print("\n📊 DistilBERT Results:")
print(f"Accuracy: {accuracy_bert:.4f}")
print(f"F1 Score: {f1_bert:.4f}")
print("\nClassification Report:\n", classification_report(y_true_bert, y_pred_bert))
print("Confusion Matrix:\n", confusion_matrix(y_true_bert, y_pred_bert))


NameError: name 'model' is not defined

In [ ]:
from torch.utils.data import DataLoader
import torch

# Make sure model is in evaluation mode
model.eval()

# Create DataLoader
test_loader = DataLoader(test_dataset, batch_size=16)

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute metrics
accuracy_bert = accuracy_score(all_labels, all_preds)
f1_bert = f1_score(all_labels, all_preds, average='weighted')

print("\n📊 DistilBERT Results:")
print(f"Accuracy: {accuracy_bert:.4f}")
print(f"F1 Score: {f1_bert:.4f}")
print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=labels_map.keys()))
print("Confusion Matrix:\n", confusion_matrix(all_labels, all_preds))


NameError: name 'model' is not defined

In [ ]:
comparison = pd.DataFrame({
    "Model": ["TF-IDF + Logistic Regression", "DistilBERT"],
    "Accuracy": [accuracy_lr, accuracy_bert],
    "F1 Score": [f1_lr, f1_bert]
})
print("\n📋 Comparison Summary:")
print(comparison)



📋 Comparison Summary:
                          Model  Accuracy  F1 Score
0  TF-IDF + Logistic Regression  0.964045  0.964028
1                    DistilBERT  0.182022  0.069677


In [ ]:
import streamlit as st
import joblib
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

# ----------------------
# Load TF-IDF + Logistic Regression model
# ----------------------
lr_model = joblib.load("logistic_regression_model.pkl")
tfidf_vectorizer = joblib.load("tfidf_vectorizer.pkl")

# ----------------------
# Load DistilBERT model
# ----------------------
bert_model = AutoModelForSequenceClassification.from_pretrained("./distilbert_model")
bert_tokenizer = AutoTokenizer.from_pretrained("./distilbert_model")

# Label mapping (update if different)
labels = ["business", "entertainment", "politics", "sport", "tech"]

st.title("📌 BBC News Classifier")
st.write("Enter a news headline or article text, and the app will predict its category.")

# User input
user_input = st.text_area("✏️ Enter your text here:")

model_choice = st.radio(
    "Choose Model:",
    ("TF-IDF + Logistic Regression", "DistilBERT")
)

if st.button("Predict"):
    if not user_input.strip():
        st.warning("Please enter some text!")
    else:
        if model_choice == "TF-IDF + Logistic Regression":
            # Vectorize and predict
            X = tfidf_vectorizer.transform([user_input])
            probs = lr_model.predict_proba(X)[0]
            pred_idx = np.argmax(probs)
            pred_label = labels[pred_idx]
            confidence = probs[pred_idx] * 100

        else:  # DistilBERT
            inputs = bert_tokenizer(user_input, return_tensors="pt", truncation=True, padding=True)
            with torch.no_grad():
                outputs = bert_model(**inputs)
                probs = torch.nn.functional.softmax(outputs.logits, dim=-1)[0].numpy()
            pred_idx = np.argmax(probs)
            pred_label = labels[pred_idx]
            confidence = probs[pred_idx] * 100

        # Display results
        st.success(f"**Predicted Category:** {pred_label}")
        st.info(f"**Confidence:** {confidence:.2f}%")
